In [14]:
import stim
import numpy as np
from mle_decoder import MLEDecoder

num_shots = 100
d = 5
p = 0.007
rounds = 3

circuit = stim.Circuit.generated(
    "surface_code:rotated_memory_x",
    rounds=rounds,
    distance=d,
    before_round_data_depolarization=p,
    before_measure_flip_probability=p,
    after_reset_flip_probability=p,
    after_clifford_depolarization=p
)

sampler = circuit.compile_detector_sampler()
shots, observables = sampler.sample(num_shots, separate_observables=True)

decoder = MLEDecoder(circuit.detector_error_model(), verbose=False)

predicted_observables = decoder.decode_batch(shots)
num_mistakes = np.sum(np.any(predicted_observables != observables, axis=1))

print(f"{num_mistakes}/{num_shots}")

ValueError: Unrecognized task 'rotated_memory_x_2'. Known surface_code tasks:
    'rotated_memory_x': Initialize logical |+> in rotated code, protect with parity measurements, measure logical X.
    'rotated_memory_z': Initialize logical |0> in rotated code, protect with parity measurements, measure logical Z.
    'unrotated_memory_x': Initialize logical |+> in unrotated code, protect with parity measurements, measure logical X.
    'unrotated_memory_z': Initialize logical |0> in unrotated code, protect with parity measurements, measure logical Z.


In [8]:
shots.shape

(100, 72)

In [9]:
circuit.num_qubits

64

In [10]:
circuit.num_detectors

72

In [13]:
circuit.num_detectors / rounds == d**2 - 1

True

是的——**num_detectors 由“码型 + 距离 d”完全决定**，不是自由参数。
下面给出常见拓扑的**一次性公式**，你代入 d 就能直接拿到 `num_detectors`（也即 stim 采样返回的 syndrome 宽度）。

------------------------------------------------
1. 标准 **rotated surface code**（stim 默认生成的）

- **X 型检测器**（Z 型 stabilizer）
  每轮数量 = (d² − 1) ⁄ 2
- **Z 型检测器**（X 型 stabilizer）
  每轮数量 = (d² − 1) ⁄ 2

⇒ **每轮总检测器**
```
det_per_round = d² − 1
```

跑 `r` 轮 stim 电路（含 r 轮 syndrome + 1 轮数据测量）→
```
num_detectors = r × (d² − 1)
```

------------------------------------------------
2. **unrotated / planar surface code**（传统 2d×2d 格点）

```
det_per_round = (d − 1)²        # 内部 (d-1)×(d-1) 个 face
num_detectors = r × (d − 1)²
```

------------------------------------------------
3. **color code** (三角蜂窝，距离 d 指“颜色边界最短链”)

```
det_per_round = 3 × (d − 1)² ⁄ 2   # 三种颜色 face
num_detectors = r × 3 × (d − 1)² ⁄ 2
```

------------------------------------------------
4. **hypergraph / LDPC**（随机 Type I/II 等）

```
det_per_round = 码的校验矩阵行数 = rank(H)
```
MCCD 里通常直接给 `mccd_circuit.n_check_nodes`，与 d 无简单闭式，但**仍由拓扑一次性固定**。

------------------------------------------------
验证 stim 实例

```python
import stim
c = stim.generated('surface_code:rotated_memory_z', distance=5, rounds=10)
print(c.num_detectors)   # 240 = 10*(5^2-1)
```

------------------------------------------------
一句话
**num_detectors = 每轮检测器数 × 轮数**
而“每轮检测器数”**只取决于码型 + 距离 d**，与噪声大小、逻辑类型都无关；
 stim 采样返回的 syndrome 宽度就是这个值。